# Hidden Number Problem

O problema do número escondido está relacionado com duas funções sobre elementos do corpo $Z_p$ e números naturais:
1. A primeira função, a **norma de um elemento em $Z_p$**, é definida da seguinte maneira:
    $\mid\mid X\mid\mid_p=$min$(x, p-x)$
1. A segunda função devolve os **$k$ bits mais significativos do seu argumento**, um número natural, módulo $p$: $msb_{k,p}(x)$

## Boneh & Venkatesan Algorithm

In [186]:
import random

class BV:
    def __init__(self, p, k, l, s):
        self.p = p
        self.k = k
        self.l = l
        self.lambd = 2 ** (k+1)
        self.s = s
        self.x = [random.randint(0, self.p-1) for i in range(0,self.l)]
        self.t = [self.lambd*self.msb(self.s*x_i) for x_i in self.x]
        self.t.append(0)
        self.gen_lattice()

    def gen_lattice(self):
        ''' Calculate the lattice base
        '''
        self.lattice = []
        
        for i in range(0,self.l):
            self.lattice.append([])
            for j in range(0, self.l):
                if i == j:
                    self.lattice[i].append(self.lambd * self.p)
                else:
                    self.lattice[i].append(0)
            
        for line in self.lattice:
            line.append(0)
            
        lambd_v_x = [self.lambd * x_i for x_i in self.x]
        self.lattice.append(lambd_v_x + [1])
    
    def msb(self, x):
        ''' Return an approximation of the k most significative bits of x
        '''
        valid_u = False
        while (not valid_u):
            u = random.randint(0, self.p - 1)
            x_u = x - u
            norm_int = min(x_u, self.p - x_u)
            valid_u = (norm_int <= self.p/self.lambd)
        return u
    
    def run_algorithm(self):
        ''' Use B&V to calculate the secret
        '''
        w = [-(self.s*x_i//self.p) for x_i in self.x]
        w.append(self.s)
        alfa = [0] * (self.l + 1)
        for i in range(0, self.l+1):
            for j in range(0, self.l+1):
                alfa[i] += w[j]*self.lattice[j][i]
        return alfa[self.l]

In [187]:
bv = BV(13,2,3,4)
bv.run_algorithm()

4